In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
MAX_SENTENCE_LENGTH = 500
import nltk
import numpy as np
import torch
from torch.utils.data import Dataset
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
from torch.autograd import Variable
from def_attn_model_word_eff import Attention
device=0

MAX_VOCAB = 50000
PAD_IDX = 0
UNK_IDX = 1

In [4]:
df_val = pd.read_csv("./processed_data/idized_hollow_val.csv")

import pickle
with open("./processed_data/token_and_id.pk",'rb') as f:
    token_2id = pickle.load(f)
    id2_token = pickle.load(f)
    all_toks = pickle.load(f)

hollow_label_encoder=pickle.load(open("./raw_data/hollow_label_encoder",'rb'))

In [14]:
tax = pd.read_csv("taxonomy_gnarly.csv")
tax['encoded_top'] = tax['topic_id'].apply(lambda x: hollow_label_encoder.index(x))

In [5]:
class HollowDataset(Dataset):
    def __init__(self, csv_file):
        self.data_frame = csv_file
    def __len__(self):
        return len(self.data_frame)
    def __getitem__(self, idx):
        token_idx = np.array(self.data_frame.iloc[idx]["idized"][1:-1].split(',')).astype(int)
        label = self.data_frame.iloc[idx]['topic_id'].astype(int)
        return [token_idx, len(token_idx), label]
def pad_fun(batch):
    data_list = []
    label_list = []
    length_list = []
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    for datum in batch:
        if datum[1]>MAX_SENTENCE_LENGTH:
            padded_vec = np.array(datum[0][:MAX_SENTENCE_LENGTH])
        else:
            padded_vec = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_SENTENCE_LENGTH - datum[1])),
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.from_numpy(np.array(length_list)), torch.from_numpy(np.array(label_list))]
BATCH_SIZE = 20

In [6]:
val_dataset = HollowDataset(df_val)
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                           batch_size = BATCH_SIZE,
                                           collate_fn = pad_fun,
                                           shuffle = True)

In [20]:
def print_attention(model_name):
    model = torch.load(model_name)
    data = next(iter(val_loader))
    x = Variable(data[0]).cuda()
    l = Variable(data[1]).cuda()
    y = Variable(data[2]).long().cuda()
    y_out,y_sm_out,w_attn,_ = model(x)
    _, top1_predicted = torch.max(y_out, dim=1)

    print("printing weights")
    n_x = x.detach().cpu().numpy()
    n_y = y.detach().cpu().numpy()
    n_p = top1_predicted.detach().cpu().numpy()
    n_w = w_attn.detach().cpu().numpy()
    n_l = l.detach().cpu().numpy()
    n_id2_tk = np.array(id2_token)
    for i in range(BATCH_SIZE):
        print("length of the doc ",n_l[i])
        print("document: "," ".join(n_id2_tk[n_x[i][:n_l[i]]]))
        print("actual class =",n_y[i],hollow_label_encoder[n_y[i]],tax[tax['encoded_top']==n_y[i]]['topic_name'])
        print("pred class =",n_p[i],hollow_label_encoder[n_p[i]],tax[tax['encoded_top']==n_p[i]]['topic_name'])
        m_i = list(np.argsort(n_w[i])[::-1][:10])
        print("Attention : ",n_id2_tk[n_x[i][m_i]])
#         print(m_i)
        print("Attention weights: ",np.sort(n_w[i])[::-1][:10])
        print("-"*100)
        print("-"*100)


In [21]:
print_attention("../attention_models/model_att5")

/home/chegde/anaconda3/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'def_attn_model_word_eff.Attention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/chegde/final_attention_model/def_attn_model_word_eff.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  out, hidden = self.gru(emb,hidden1)
/home/chegde/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


printing weights
length of the doc  864
document:  deep dive culture culture amp ceo didier elzinga didier elzinga ceo employee feedback platform culture amp, solution allows businesses take pulse workers take action quickly, company culture isn’t important aspect <unk> primary focus. it’s important remember there’s no <unk> solution solve company culture puzzle organizations, according <unk> “there’s no thing perfect <unk> elzinga says. “it’s building culture appropriate type company you’re building. way think that, it’s done right, good values make individual want better version <unk> culture amp draws data questions content across 1,600 customers <unk> community that’s almost 50,000 strong. last month, company announced secured <unk> series d funding, bringing culture <unk> total amount equity funding <unk> since founded eight years ago. elzinga told chief executive importance employee feedback ceos, manage relationships outside investors he’s managing company’s culture. here’s say: